In [1]:
%load_ext autoreload
%autoreload 2
from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.io.fits as fits
import alipy
import numpy as np
import os
import glob
import datetime as dt
import processimages as pim
!cd /home/pi/DATA/astro/suhora/pol_survey

In [19]:
def create_im_list(main_dir, start_date, end_date, ext='*red.fit'):
    container = []
    for root, dirs, files in os.walk(main_dir, topdown=False):
        for name in dirs:
            folder_root_name = os.path.join(root, name)
            # format w zaleznosci od systemu
            # folder_name = folder_root_name.split('\\')[-1]
            folder_name = folder_root_name.split('/')[-1]
            try:
                f_date = dt.datetime.strptime(folder_name, '%Y-%m-%d')
                if (f_date >= start_date and f_date <= end_date):
                    folder_content = sorted(glob.glob(
                        os.path.join(folder_root_name, ext)))
                    if len(folder_content) > 0:
                        container.append([folder_root_name, folder_content])
            except ValueError:
                pass
            
    return container


def select_exp(pack, exp, files_ext):
    express = ''.join(['-', str(exp), files_ext])

    return list((filter(lambda im: express in im, pack)))


def select_filter(pack, fil):
    
    return list(filter(lambda im: fil in im, pack))


def create_coo_boxes(pack, radius, unit):
    
    radius = radius * getattr(u, unit)
    images_fields = []
    grouped_fields = []
    
    for im in pack:
        im_name = os.path.basename(im)
        # tutaj trzeba uwazac jezeli zmieni sie format nazwy
        im_field = im_name.split('_')[1]
        
        if '+' in im_field:
            im_ra, im_dec = im_field.split('+')
        elif '-' in im_field:
            im_ra, im_dec = im_field.split('-')
        else:
            print('Something wrong with coordinates format in im name')
            break
            
        # one line, slightly complicated solution for change HHMMSS to HH:MM:SS 
        im_ra = ':'.join(list(map(''.join, zip(*[iter(im_ra)]*2))))
        im_dec = ':'.join(list(map(''.join, zip(*[iter(im_dec)]*2))))
        im_field = SkyCoord(ra=im_ra, dec=im_dec, unit=(u.hourangle, u.deg))
        images_fields.append([im, im_field])
        
    for im_field in images_fields:
        group = [f[0] for f in images_fields if f[1].separation(im_field[1]) < radius]
        if len(group) > 0: grouped_fields.append(group)
        images_fields = [f for f in images_fields if f[0] not in group]
        
    return grouped_fields

In [20]:
# create_im_list('/home/pi/DATA/astro/suhora/pol_survey/', dt.datetime(2015, 1, 1), dt.datetime(2020, 12, 31))

In [27]:
def prepare_stack(main_dir, save_dir, start_date, end_date, files_ext='_red.fit',
                  ext='*red.fit', exps=[5, 60], 
                  filters=['P1', 'P2', 'P3', 'P4'], 
                  radius=4, unit='deg'):
    try:
        os.mkdir(save_dir)
    except FileExistsError:
        pass
    
    container = create_im_list(main_dir, start_date, 
                               end_date, ext=ext)
    
    for pack_date, pack in container:
        for exp in exps:
            selected_exp = select_exp(pack, exp, files_ext)
            if selected_exp:
                coo_boxes = create_coo_boxes(selected_exp, radius, unit)
                for coo_box in coo_boxes:
                    stacked_images = []
                    for filter_name in filters:
                        filter_pack = select_filter(coo_box, filter_name)
                        pim.align_images(filter_pack)
                        stacked_image = pim.make_stack(filter_pack, save_dir, exp, filter_name)
                        stacked_images.append(stacked_image)
                    pim.align_images(stacked_images)

In [ ]:
prepare_stack(main_dir='/home/pi/DATA/astro/suhora/pol_survey/',
              save_dir='/home/pi/DATA/astro/suhora/pol_survey/stacked_fields/',
              start_date=dt.datetime(2010, 1, 1),
              end_date=dt.datetime(2020, 12, 31))